<a href="https://colab.research.google.com/github/BamByeol/learning-ML/blob/main/FinalTerm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#csv 파일 불러오기 : star type을 예측하는 star dataset
#자료출처 ( https://www.kaggle.com/datasets/deepu1109/star-dataset?resource=download )

df = pd.read_csv('/content/drive/MyDrive/6 class csv.csv')
df.head()
df.info()

In [ ]:
#데이터 전저리
#df['Spectral Class'] = df['Spectral Class'].astype(int)
#df['Star color'].value_counts()

#star color dtype 전처리 : RED -> 0 / BLUE -> 1 / Others -> 2

#print(df)
#df.replace({'col_1':{1:100},'col_2':{2:200}})

#df['Star color'].replace({0:'Red', 1:'Blue', 2:''},inplace=True)


#df.dtypes
df['Spectral Class'].value_counts()

df.drop('Spectral Class', axis=1, inplace= True)

#df['Star color'].replace('Red',0,inplace=True)
#df['Star color'].replace('Blue',1,inplace=True)
#df['Star color'].replace('Red',0,inplace=True)
#df['Star color'].replace('Blue',1,inplace=True)

#Idx = df[df['Star color'] != 'Red'].index
#df.drop(Idx , inplace=True)

#Idx2 = df[df['Star color'] != 'Blue'].index
#df.drop(Idx2 , inplace=True)

#def color():
#    if 'Star type' != 'Red':
#        Idx = df[df['Star color'] != 'Red'].index
#        df.drop(Idx , inplace=True)

#    elif 'Star type' != 'Blue':
#        Idx2 = df[df['Star color'] != 'Blue'].index
#        df.drop(Idx2 , inplace=True)         

    #elif 'Star type' == 'Red' or 'Star type' == 'Blue':
        #df['Star color'].replace('Red',0,inplace=True)
        #df['Star color'].replace('Blue',1,inplace=True)


#color()


#df['Star color'].value_counts()

#df['Star color'] = df['Star color'].astype('int')

#df.info()

In [ ]:
df.info()

In [ ]:
# XGBoost

from sklearn.model_selection import train_test_split

y_target = df['Star type']
X_features = df.drop(['Star type'], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, random_state=10, test_size = 0.2)


from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

def evaluate(y_test, y_hat, y_prob):
    print(accuracy_score(y_test, y_hat))
#    print(mean_squared_error(y_test, y_prob))
#    print(f1_score(y_test, y_hat, average='samples'))
#    print(roc_auc_score(y_test,y_prob))

xgb = XGBClassifier(n_estimators = 600, learning_rate=0.02)
xgb.fit(X_train,y_train,early_stopping_rounds=300, eval_metric='mlogloss', eval_set=[(X_train, y_train),(X_test,y_test)])
y_hat = xgb.predict(X_test)
y_prob = xgb.predict_proba(X_test)[:,1]

#oc_score = roc_auc_score(y_test, y_prob, multi_class='ovo')
evaluate(y_test,y_hat,y_prob)
#mse = mean_squared_error(y_true, y_pred) 
#print(roc_score)
#print(accuracy_score(y_test, y_hat))


from xgboost import plot_importance
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1,figsize=(10,8))
plot_importance(xgb, ax=ax, max_num_features=20, height=0.4)

In [ ]:
#random forest

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=10)
clf.fit(X_train, y_train)

features = df.drop(['Star type'], axis = 1)
labels = df['Star type']
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_train_features = scaler.fit_transform(features)
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(scaled_train_features)
pca_projection = pca.transform(scaled_train_features)


from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=10)

#tree = DecisionTreeClassifier()
#logreg = LogisticRegression()
clf = RandomForestClassifier()

#tree_score = cross_val_score(tree, pca_projection, labels, cv=kf)
#logit_score = cross_val_score(logreg, pca_projection, labels, cv=kf)
rt_score = cross_val_score(clf,pca_projection, labels, cv=kf)

# Mean of all the score arrays
print("Random Forest:",np.mean(rt_score))

In [ ]:
#LightGBM
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators=600, learning_rate=0.02)

lgbm.fit(X_train, y_train, early_stopping_rounds=50, eval_set=[(X_test, y_test)])
y_hat=lgbm.predict(X_test)
y_prob=lgbm.predict_proba(X_test)[:1]

def evaluate_lgbm(y_test, y_hat, y_prob):
    print(accuracy_score(y_test, y_hat))

evaluate_lgbm(y_test, y_hat, y_prob)